In [11]:
from playwright.async_api import async_playwright

async def capture_endpoints():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        endpoints = set()

        page.on(
            "response",
            lambda res: endpoints.add(res.url)
            if "/webapi/api/" in res.url else None
        )

        await page.goto("https://financiamento.iapmei.pt/inicio/home", wait_until="domcontentloaded")

        # ⬇️ THIS IS THE IMPORTANT PART
        await page.wait_for_timeout(3000)
        await page.mouse.wheel(0, 3000)   # trigger lazy loading
        await page.wait_for_timeout(3000)

        await browser.close()
        return endpoints


In [12]:
endpoints = await capture_endpoints()
for e in endpoints:
    print(e)


https://financiamento.iapmei.pt/webapi/api/ItensPesquisas?caraterizacao=2
https://financiamento.iapmei.pt/webapi/api/Paginas/Footer
https://financiamento.iapmei.pt/webapi/api/indicadores
https://financiamento.iapmei.pt/webapi/api/ItensPesquisas/arrayvalores
https://financiamento.iapmei.pt/webapi/api/paginas
https://financiamento.iapmei.pt/webapi/api/sliders
https://financiamento.iapmei.pt/webapi/api/ItensPesquisas
https://financiamento.iapmei.pt/webapi/api/ItensPesquisas?caraterizacao=1
https://financiamento.iapmei.pt/webapi/api/Itens/arrayvalores
https://financiamento.iapmei.pt/webapi/api/Configuracao/GetRedesSociais


In [18]:
import asyncio
from playwright.async_api import async_playwright
import requests
from bs4 import BeautifulSoup
import json
from pathlib import Path

OUTPUT_DIR = Path("data")
OUTPUT_DIR.mkdir(exist_ok=True)

BASE_URL = "https://financiamento.iapmei.pt/webapi/api"

# ------------------------------
# Helper functions
# ------------------------------
def clean_html(html_str):
    if not html_str:
        return ""
    return BeautifulSoup(html_str, "html.parser").get_text(separator="\n", strip=True)

def fetch_json(url, params=None):
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

def extract_text_from_data(data):
    texts = []

    if isinstance(data, dict):
        for key in ["titulo", "descricao", "conteudo", "texto"]:
            val = data.get(key)
            if isinstance(val, str) and val.strip():
                texts.append(clean_html(val))

        # Rich text components
        for comp in data.get("componentes", []):
            if isinstance(comp, dict):
                for v in comp.values():
                    if isinstance(v, str) and len(v.strip()) > 50:
                        texts.append(clean_html(v))

    elif isinstance(data, list):
        for el in data:
            if isinstance(el, dict):
                for v in el.values():
                    if isinstance(v, str) and len(v.strip()) > 50:
                        texts.append(clean_html(v))
            elif isinstance(el, str) and el.strip():
                texts.append(clean_html(el))

    return "\n".join(texts)

# ------------------------------
# Step 1: Capture endpoints via Playwright
# ------------------------------
async def capture_endpoints():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # set True to hide browser
        page = await browser.new_page()
        endpoints = set()

        # Capture all API responses
        page.on("response", lambda res: endpoints.add(res.url) if "/webapi/api/" in res.url else None)

        # Open homepage
        await page.goto("https://financiamento.iapmei.pt/inicio/home", wait_until="domcontentloaded")

        # Trigger lazy-loaded content
        await page.wait_for_timeout(3000)
        await page.mouse.wheel(0, 3000)
        await page.wait_for_timeout(3000)

        await browser.close()
        return endpoints

# ------------------------------
# Step 2: Fetch content from discovered endpoints
# ------------------------------
async def main():
    endpoints = await capture_endpoints()
    print(f"Discovered {len(endpoints)} API endpoints")
    
    dataset = []

    # Filter relevant endpoints: only Itens / Paginas / ItensPesquisas
    relevant = [e for e in endpoints if any(x in e.lower() for x in ["/itens", "/paginas", "/itenspesquisas"])]
    print(f"Processing {len(relevant)} relevant endpoints")

    for url in relevant:
        try:
            data = fetch_json(url)
            text = extract_text_from_data(data)

            dataset.append({
                "url": url,
                "text": text
            })
            print(f"Fetched {url} → {len(text)} chars")

        except Exception as e:
            print(f"Error fetching {url}: {e}")

    # Save final dataset
    output_file = OUTPUT_DIR / "financiamento_complete.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)

    print(f"All content saved to {output_file}")
    print(f"Total documents: {len(dataset)}")

# ------------------------------
# Run
# ------------------------------
import nest_asyncio
nest_asyncio.apply()

import asyncio

# Instead of asyncio.run(main()), do:
await main()


Discovered 10 API endpoints
Processing 7 relevant endpoints
Fetched https://financiamento.iapmei.pt/webapi/api/ItensPesquisas?caraterizacao=2 → 0 chars
Fetched https://financiamento.iapmei.pt/webapi/api/Paginas/Footer → 7578 chars
Fetched https://financiamento.iapmei.pt/webapi/api/ItensPesquisas/arrayvalores → 0 chars
Fetched https://financiamento.iapmei.pt/webapi/api/paginas → 11209 chars
Fetched https://financiamento.iapmei.pt/webapi/api/ItensPesquisas → 0 chars
Fetched https://financiamento.iapmei.pt/webapi/api/ItensPesquisas?caraterizacao=1 → 0 chars
Fetched https://financiamento.iapmei.pt/webapi/api/Itens/arrayvalores → 0 chars
All content saved to data/financiamento_complete.json
Total documents: 7


In [39]:
import asyncio
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import json
from playwright.async_api import async_playwright
import nest_asyncio

# ------------------------------
# Settings
# ------------------------------
OUTPUT_DIR = Path("data")
OUTPUT_DIR.mkdir(exist_ok=True)

SITES = [
    "https://financiamento.iapmei.pt/inicio/home",
    "https://transparencia.gov.pt/pt/fundos-europeus/pt2030/avisos/"
]

# ------------------------------
# Helper functions
# ------------------------------
def clean_html(html_str):
    if not html_str:
        return ""
    return BeautifulSoup(html_str, "html.parser").get_text(separator="\n", strip=True)

def fetch_json(url, params=None):
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

def extract_all_text(data):
    """Recursively extract all meaningful strings from JSON"""
    texts = []

    if isinstance(data, dict):
        for v in data.values():
            texts.extend(extract_all_text(v))
    elif isinstance(data, list):
        for el in data:
            texts.extend(extract_all_text(el))
    elif isinstance(data, str) and len(data.strip()) > 20:
        texts.append(clean_html(data))

    return texts

def is_json_endpoint(url):
    """Optional: check if endpoint returns JSON"""
    try:
        r = requests.head(url, timeout=5)
        return "application/json" in r.headers.get("Content-Type", "")
    except:
        return False

# ------------------------------
# Capture endpoints via Playwright
# ------------------------------
async def capture_endpoints(site_url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        endpoints = set()

        # Capture all API responses
        page.on("response", lambda res: endpoints.add(res.url) if "api" in res.url.lower() else None)

        await page.goto(site_url, wait_until="domcontentloaded")

        # Scroll a bit to trigger lazy-loaded requests
        for _ in range(3):
            await page.mouse.wheel(0, 2000)
            await page.wait_for_timeout(1000)

        await browser.close()
        return endpoints

# ------------------------------
# Process one website
# ------------------------------
async def process_site(site_url):
    print(f"Processing site: {site_url}")
    endpoints = await capture_endpoints(site_url)
    print(f"  Discovered {len(endpoints)} endpoints")

    dataset = []
    for ep in endpoints:
        # Optional: skip non-JSON endpoints
        # if not is_json_endpoint(ep):
        #     continue

        try:
            data = fetch_json(ep)
            text = "\n".join(extract_all_text(data))
            if text.strip():
                dataset.append({"url": ep, "text": text})
                print(f"  Fetched {ep} → {len(text)} chars")
        except Exception as e:
            print(f"  Error fetching {ep}: {e}")

    return dataset

# ------------------------------
# Main function
# ------------------------------
async def main():
    all_data = await asyncio.gather(*(process_site(site) for site in SITES))
    flat_data = [item for sublist in all_data for item in sublist]

    output_file = OUTPUT_DIR / "dataset.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(flat_data, f, ensure_ascii=False, indent=2)

    print(f"\nSaved {len(flat_data)} documents to {output_file}")

# ------------------------------
# Run
# ------------------------------
nest_asyncio.apply()
await main()


Processing site: https://financiamento.iapmei.pt/inicio/home
Processing site: https://transparencia.gov.pt/pt/fundos-europeus/pt2030/avisos/
  Discovered 0 endpoints
  Discovered 12 endpoints
  Fetched https://financiamento.iapmei.pt/webapi/api/ItensPesquisas?caraterizacao=2 → 4398 chars
  Fetched https://financiamento.iapmei.pt/webapi/api/Paginas/Footer → 7787 chars
  Error fetching https://region1.analytics.google.com/g/collect?v=2&tid=G-4VGZT4V745&gtm=45je5ca1v9108577613za20g&_p=1766571191729&_gaz=1&gcd=13l3l3l2l1l1&npa=1&dma_cps=syphamo&dma=1&cid=334226070.1766571192&ul=en-us&sr=1280x720&ir=1&uaa=x86&uab=64&uafvl=HeadlessChrome%3B143.0.7499.4%7CChromium%3B143.0.7499.4%7CNot%2520A(Brand%3B24.0.0.0&uamb=0&uam=&uap=macOS&uapv=26.1.0&uaw=0&are=1&frm=0&pscdl=noapi&_eu=EAAI&_s=1&tag_exp=103116026~103200004~104527906~104528500~104684208~104684211~105391252~115583767~115938465~115938469~116184927~116184929~116251938~116251940&sid=1766571192&sct=1&seg=0&dl=https%3A%2F%2Ffinanciamento.iapmei

/var/folders/mm/fxsq_1490x9dd2w76tqvt3kr0000gn/T/ipykernel_2691/4261549396.py:26: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  return BeautifulSoup(html_str, "html.parser").get_text(separator="\n", strip=True)


  Fetched https://financiamento.iapmei.pt/webapi/api/ItensPesquisas → 10095 chars
  Fetched https://financiamento.iapmei.pt/webapi/api/ItensPesquisas?caraterizacao=1 → 5025 chars
  Fetched https://financiamento.iapmei.pt/webapi/api/Itens/arrayvalores → 6322 chars
  Fetched https://financiamento.iapmei.pt/webapi/api/Configuracao/GetRedesSociais → 190 chars

Saved 10 documents to data/dataset.json
